# Hospital Facilities Insight 

Data obtained from Community Benefits Insight: http://www.communitybenefitinsight.org/api/get_hospitals.php

## Import Libaries

In [1]:
# import appropriate packages
import os
import requests
import json 
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql.cursors
from pymysql import IntegrityError

import powerbiclient

from powerbiclient import QuickVisualize, get_dataset_config, Report
from powerbiclient.authentication import DeviceCodeLoginAuthentication

from geopy.geocoders import Bing
import folium

import warnings
warnings.filterwarnings("ignore") #supress warnings

/var/folders/7x/mfyhhlkn0fz774bmn50_szsm0000gn/T/ipykernel_23358/623770789.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Import Data

In [2]:
# API endpoint
api_url = 'http://www.communitybenefitinsight.org/api/get_hospitals.php'

# requesting data from API
response = requests.get(api_url)

# check status
if response.status_code == 200:
    data = response.json()

    # data update check
    last_modified_header = response.headers.get('Last-Modified')

    if last_modified_header:
        last_modified = datetime.strptime(last_modified_header, '%a, %d %b %Y %H:%M:%S GMT')
        print(f"Data was last modified: {last_modified}")

    # convert data to dataframe in chunks
    chunk_size = 100  # define size of chunk
    hospital_dfs = []  # create list to store df 

    for chunk_start in range(0, len(data), chunk_size):
        chunk_end = min(chunk_start + chunk_size, len(data))
        chunk_data = data[chunk_start:chunk_end]
        hospital_df = pd.DataFrame(chunk_data)
        hospital_dfs.append(hospital_df)

    # concate all df chunks into one
    hospital_df = pd.concat(hospital_dfs, ignore_index=True)

    # print df 
    print(hospital_df)
    
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")
    print(response.text)

     hospital_id hospital_org_id        ein  \
0              1               1  630307951   
1              2               2  630578923   
2              3               3  630312913   
3              4               4  630459034   
4              5               5  581973570   
...          ...             ...        ...   
3486        3487            2647  813040663   
3487        3488            2304  741109643   
3488        3489            2648  831954982   
3489        3490            2302  750800661   
3490        3491            2649  831869297   

                                     name  \
0                Mizell Memorial Hospital   
1                        St Vincents East   
2           Shelby Baptist Medical Center   
3            Callahan Eye Foundation Hosp   
4                 Cherokee Medical Center   
...                                   ...   
3486          Bsw Medical Center - Austin   
3487              Ascension Seton Bastrop   
3488         Texas Health Hosp

In [3]:
hospital_df # print df 

,hospital_id,hospital_org_id,ein,name,name_cr,street_address,city,state,zip_code,fips_state_and_county_code,hospital_bed_count,chrch_affl_f,urban_location_f,children_hospital_f,memb_counc_teach_hosps_f,medicare_provider_number,county,hospital_bed_size,updated_dt
0,1,1,630307951,Mizell Memorial Hospital,Mizell Memorial Hospital,702 Main Street,Opp,AL,36462,01039,99,N,N,N,N,010007,Covington County,<100 beds,"November 20, 2023"
1,2,2,630578923,St Vincents East,St Vincents East,50 Medical Park Drive East,Birmingham,AL,35235,01073,362,N,Y,N,Y,010011,Jefferson County,>299 beds,"November 20, 2023"
2,3,3,630312913,Shelby Baptist Medical Center,Shelby Baptist Medical Center,1000 First Street North,Alabaster,AL,35007,01117,252,N,Y,N,N,010016,Shelby County,100-299 beds,"November 20, 2023"
3,4,4,630459034,Callahan Eye Foundation Hosp,Callahan Eye Foundation Hosp,1720 University Boulevard,Birmingham,AL,35233,01073,106,N,Y,N,Y,010018,Jefferson County,100-299 beds,"November 20, 2023"
4,5,5,581973570,Cherokee Medical Center,Cherokee Medical Center,400 Northwood Drive,Centre,AL,35960,01019,60,N,N,N,N,010022,Cherokee County,<100 beds,"November 20, 2023"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3486,3487,2647,813040663,Bsw Medical Center - Austin,Bsw Medical Center - Austin,5245 W Us 290,Austin,TX,78735,48453,16,N,Y,N,N,670136,Travis County,<100 beds,"November 20, 2023"
3487,3488,2304,741109643,Ascension Seton Bastrop,Ascension Seton Bastrop,630 Highway 71 W,Bastrop,TX,78602,48021,7,N,Y,N,N,670143,Bastrop County,<100 beds,"November 20, 2023"
3488,3489,2648,831954982,Texas Health Hospital Frisco,Texas Health Hospital Frisco,12400 N Dallas Parkway,Frisco,TX,75033,48121,63,N,Y,N,N,670260,Denton County,<100 beds,"November 20, 2023"
3489,3490,2302,750800661,Methodist Midlothian Medical Center,Methodist Midlothian Medical Center,1201 E Highway 287,Midlothian,TX,76065,48139,46,N,Y,N,N,670300,Ellis County,<100 beds,"November 20, 2023"


In [4]:
# print 10 random samples from the dataframe 
hospital_df.sample(10)

,hospital_id,hospital_org_id,ein,name,name_cr,street_address,city,state,zip_code,fips_state_and_county_code,hospital_bed_count,chrch_affl_f,urban_location_f,children_hospital_f,memb_counc_teach_hosps_f,medicare_provider_number,county,hospital_bed_size,updated_dt
3145,3146,850,521301088,Bon Secours Depaul Medical Center,,150 Kingsley Lane,Norfolk,VA,23505,51710,204,N,Y,N,Y,490011,Norfolk city,100-299 beds,"November 20, 2023"
416,417,263,840405257,Centura Mercy Hospital,Centura Mercy Hospital,1010 Three Springs Blvd,Durango,CO,81301,08067,82,Y,N,N,Y,060013,La Plata County,<100 beds,"November 20, 2023"
467,468,292,060646704,Lawrence & Memorial Hospital,Lawrence & Memorial Hospital,365 Montauk Avenue,New London,CT,06320,09011,280,N,Y,N,N,070007,New London County,100-299 beds,"November 20, 2023"
1887,1888,1475,222674014,Exeter Hospital Inc,Exeter Hospital Inc,5 Alumni Drive,Exeter,NH,03833,33015,100,N,Y,N,N,300023,Rockingham County,100-299 beds,"November 20, 2023"
2568,2569,256,362193608,Shriners Hospitals For Children,Shriners Hospitals For Children,3101 Sw Sam Jackson Park Road,Portland,OR,97239,41051,29,N,Y,Y,Y,383300,Multnomah County,<100 beds,"November 20, 2023"
3466,3467,2349,752702388,Presbyterian Hosp Flower Mound,Presbyterian Hosp Flower Mound,4400 Long Prairie Road,Flower Mound,TX,75028,48121,99,N,Y,N,N,670068,Denton County,<100 beds,"November 20, 2023"
1735,1736,1344,431936696,Mercy Hospital Aurora,Mercy Hospital Aurora,500 Porter Street,Aurora,MO,65605,29109,25,Y,N,N,N,261316,Lawrence County,<100 beds,"November 20, 2023"
2467,2468,1951,730579285,Mercy Hospital Okc,Mercy Hospital Okc,4300 West Memorial Road,Oklahoma City,OK,73120,40109,385,Y,Y,N,N,370013,Oklahoma County,>299 beds,"November 20, 2023"
408,409,30,450233470,North Colorado Medical Center,North Colorado Medical Center,1801 16th Street,Greeley,CO,80631,08123,378,N,Y,N,N,060001,Weld County,>299 beds,"November 20, 2023"
26,27,22,630931008,St Vincents St Clair,St Vincents St Clair,7063 Veterans Parkway,Pell City,AL,35125,01115,40,N,Y,N,N,010130,St. Clair County,<100 beds,"November 20, 2023"


## Data Inspection

In [5]:
hospital_df.shape 

(3491, 19)

In [6]:
hospital_df.info() #all variables in df are labeled as object variables

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3491 entries, 0 to 3490
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   hospital_id                 3491 non-null   object
 1   hospital_org_id             3491 non-null   object
 2   ein                         3491 non-null   object
 3   name                        3491 non-null   object
 4   name_cr                     3491 non-null   object
 5   street_address              3491 non-null   object
 6   city                        3491 non-null   object
 7   state                       3491 non-null   object
 8   zip_code                    3491 non-null   object
 9   fips_state_and_county_code  3491 non-null   object
 10  hospital_bed_count          3491 non-null   object
 11  chrch_affl_f                3491 non-null   object
 12  urban_location_f            3491 non-null   object
 13  children_hospital_f         3491 non-null   obje

In [7]:
columns_to_convert = ['hospital_bed_count', 'medicare_provider_number']
binary_columns = ['chrch_affl_f', 'urban_location_f', 'children_hospital_f', 'memb_counc_teach_hosps_f']

# iterate through the list of columns
for col in columns_to_convert:
    # convert to numeric, coerce errors to NaN
    hospital_df[col] = pd.to_numeric(hospital_df[col], errors='coerce')
    
for col in binary_columns:
    # Map 'N' to 0 and 'Y' to 1
    hospital_df[col] = hospital_df[col].map({'N': 0, 'Y': 1})

# convert updated_dt to date type
hospital_df['updated_dt'] = hospital_df['updated_dt'].apply(lambda x: datetime.strptime(x, '%B %d, %Y').strftime('%Y-%m-%d'))

# check the data types after conversion
hospital_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3491 entries, 0 to 3490
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   hospital_id                 3491 non-null   object
 1   hospital_org_id             3491 non-null   object
 2   ein                         3491 non-null   object
 3   name                        3491 non-null   object
 4   name_cr                     3491 non-null   object
 5   street_address              3491 non-null   object
 6   city                        3491 non-null   object
 7   state                       3491 non-null   object
 8   zip_code                    3491 non-null   object
 9   fips_state_and_county_code  3491 non-null   object
 10  hospital_bed_count          3491 non-null   int64 
 11  chrch_affl_f                3491 non-null   int64 
 12  urban_location_f            3491 non-null   int64 
 13  children_hospital_f         3491 non-null   int6

In [8]:
hospital_df.nunique() #unique variables 

hospital_id                   3491
hospital_org_id               2377
ein                           2377
name                          3330
name_cr                       3072
street_address                3439
city                          2192
state                           51
zip_code                      3200
fips_state_and_county_code    1751
hospital_bed_count             696
chrch_affl_f                     2
urban_location_f                 2
children_hospital_f              2
memb_counc_teach_hosps_f         2
medicare_provider_number      3491
county                        1195
hospital_bed_size                3
updated_dt                       1
dtype: int64

In [9]:
hospital_df.isnull().sum() #null values 

hospital_id                   0
hospital_org_id               0
ein                           0
name                          0
name_cr                       0
street_address                0
city                          0
state                         0
zip_code                      0
fips_state_and_county_code    0
hospital_bed_count            0
chrch_affl_f                  0
urban_location_f              0
children_hospital_f           0
memb_counc_teach_hosps_f      0
medicare_provider_number      0
county                        1
hospital_bed_size             0
updated_dt                    0
dtype: int64

In [10]:
duplicate_rows = hospital_df[hospital_df.duplicated()]

print("Duplicate Rows except first occurrence:")
print(duplicate_rows)

if hospital_df.duplicated().any():
    print("Duplicates exist in the DataFrame.")
else:
    print("No duplicates found in the DataFrame.")

Duplicate Rows except first occurrence:
Empty DataFrame
Columns: [hospital_id, hospital_org_id, ein, name, name_cr, street_address, city, state, zip_code, fips_state_and_county_code, hospital_bed_count, chrch_affl_f, urban_location_f, children_hospital_f, memb_counc_teach_hosps_f, medicare_provider_number, county, hospital_bed_size, updated_dt]
Index: []
No duplicates found in the DataFrame.


## Table Manipulation

In [11]:
# connect to database

username = "jeremiahf24"
password = "$ads507password"
hostName = "ads507finalproject.mysql.database.azure.com"

conn = pymysql.connect(host=hostName,
                       port=3306,
                       user=username,
                       passwd=password, 
                       db= "ads507final", 
                       ssl={"fake_flag_to_enable_tls": True})

In [12]:
# Hospital Information Table

# create unique ID for address using the max value from table
cursor = conn.cursor()
query = "SELECT MAX(Address_id) FROM address_location"
cursor.execute(query)


max_value = cursor.fetchone()[0]
hospital_df['address_id'] = range(max_value + 1, max_value + len(hospital_df) + 1)

#create unique ID for hospital capacity
query = "SELECT MAX(capacity_id) FROM hospital_capacity"
cursor.execute(query)


max_value = cursor.fetchone()[0]
hospital_df['capacity_id'] = range(max_value + 1, max_value + len(hospital_df) + 1)

# create hospital information table
hospital = hospital_df[['hospital_id', 'hospital_org_id', 'name', 'ein', 'medicare_provider_number', 'address_id', 'capacity_id', 'updated_dt']]
hospital

,hospital_id,hospital_org_id,name,ein,medicare_provider_number,address_id,capacity_id,updated_dt
0,1,1,Mizell Memorial Hospital,630307951,10007,27929,27929,2023-11-20
1,2,2,St Vincents East,630578923,10011,27930,27930,2023-11-20
2,3,3,Shelby Baptist Medical Center,630312913,10016,27931,27931,2023-11-20
3,4,4,Callahan Eye Foundation Hosp,630459034,10018,27932,27932,2023-11-20
4,5,5,Cherokee Medical Center,581973570,10022,27933,27933,2023-11-20
...,...,...,...,...,...,...,...,...
3486,3487,2647,Bsw Medical Center - Austin,813040663,670136,31415,31415,2023-11-20
3487,3488,2304,Ascension Seton Bastrop,741109643,670143,31416,31416,2023-11-20
3488,3489,2648,Texas Health Hospital Frisco,831954982,670260,31417,31417,2023-11-20
3489,3490,2302,Methodist Midlothian Medical Center,750800661,670300,31418,31418,2023-11-20


In [13]:
# Church Affiliation Table
church_affiliation = hospital_df[['hospital_id', 'chrch_affl_f', 'updated_dt']]
church_affiliation

,hospital_id,chrch_affl_f,updated_dt
0,1,0,2023-11-20
1,2,0,2023-11-20
2,3,0,2023-11-20
3,4,0,2023-11-20
4,5,0,2023-11-20
...,...,...,...
3486,3487,0,2023-11-20
3487,3488,0,2023-11-20
3488,3489,0,2023-11-20
3489,3490,0,2023-11-20


In [14]:
# Hospital Capacity Table extracting data from hospital_df
hospital_capacity = hospital_df[['capacity_id', 'hospital_bed_size', 'hospital_bed_count', 'updated_dt']]
hospital_capacity

,capacity_id,hospital_bed_size,hospital_bed_count,updated_dt
0,27929,<100 beds,99,2023-11-20
1,27930,>299 beds,362,2023-11-20
2,27931,100-299 beds,252,2023-11-20
3,27932,100-299 beds,106,2023-11-20
4,27933,<100 beds,60,2023-11-20
...,...,...,...,...
3486,31415,<100 beds,16,2023-11-20
3487,31416,<100 beds,7,2023-11-20
3488,31417,<100 beds,63,2023-11-20
3489,31418,<100 beds,46,2023-11-20


In [15]:
# Children Hospital Table - extracting data from hospital_df
children_hospital= hospital_df[['hospital_id', 'children_hospital_f', 'updated_dt']]
children_hospital

,hospital_id,children_hospital_f,updated_dt
0,1,0,2023-11-20
1,2,0,2023-11-20
2,3,0,2023-11-20
3,4,0,2023-11-20
4,5,0,2023-11-20
...,...,...,...
3486,3487,0,2023-11-20
3487,3488,0,2023-11-20
3488,3489,0,2023-11-20
3489,3490,0,2023-11-20


In [16]:
# Teaching Table - extracting data from hospital_df
teaching = hospital_df[['hospital_id', 'memb_counc_teach_hosps_f', 'updated_dt']]
teaching

,hospital_id,memb_counc_teach_hosps_f,updated_dt
0,1,0,2023-11-20
1,2,1,2023-11-20
2,3,0,2023-11-20
3,4,1,2023-11-20
4,5,0,2023-11-20
...,...,...,...
3486,3487,0,2023-11-20
3487,3488,0,2023-11-20
3488,3489,0,2023-11-20
3489,3490,0,2023-11-20


In [17]:
# Urban Table
urban = hospital_df[['urban_location_f', 'hospital_id', 'updated_dt']]
urban

,urban_location_f,hospital_id,updated_dt
0,0,1,2023-11-20
1,1,2,2023-11-20
2,1,3,2023-11-20
3,1,4,2023-11-20
4,0,5,2023-11-20
...,...,...,...
3486,1,3487,2023-11-20
3487,1,3488,2023-11-20
3488,1,3489,2023-11-20
3489,1,3490,2023-11-20


In [18]:
# Address/Location Table
Address = hospital_df[['address_id', 'street_address','city','state','zip_code','fips_state_and_county_code','county','updated_dt']]
Address

,address_id,street_address,city,state,zip_code,fips_state_and_county_code,county,updated_dt
0,27929,702 Main Street,Opp,AL,36462,01039,Covington County,2023-11-20
1,27930,50 Medical Park Drive East,Birmingham,AL,35235,01073,Jefferson County,2023-11-20
2,27931,1000 First Street North,Alabaster,AL,35007,01117,Shelby County,2023-11-20
3,27932,1720 University Boulevard,Birmingham,AL,35233,01073,Jefferson County,2023-11-20
4,27933,400 Northwood Drive,Centre,AL,35960,01019,Cherokee County,2023-11-20
...,...,...,...,...,...,...,...,...
3486,31415,5245 W Us 290,Austin,TX,78735,48453,Travis County,2023-11-20
3487,31416,630 Highway 71 W,Bastrop,TX,78602,48021,Bastrop County,2023-11-20
3488,31417,12400 N Dallas Parkway,Frisco,TX,75033,48121,Denton County,2023-11-20
3489,31418,1201 E Highway 287,Midlothian,TX,76065,48139,Ellis County,2023-11-20


In [19]:
tableNames=pd.read_sql_query("""SHOW TABLES""", conn)

# print table names 
tableNames

,Tables_in_ads507final
0,address_location
1,children_hospital
2,church_affiliation
3,hospital
4,hospital_bed_count_by_state
5,hospital_capacity
6,hospital_facility_count_by_state
7,teaching
8,urban


In [20]:
# Address/Location table - convert df to list of tuples
address_data = [tuple(x) for x in Address.values]

# SQL query - insert data into address table
sql_insert_address_data = """
INSERT INTO address_location (Address_id, Street_address, City, State, Zip_code, fips_state_and_county_code, county, Updated_dt)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

# execute query 
if address_data:
    # Execute query 
    cursor = conn.cursor()
    try:
        cursor.executemany(sql_insert_address_data, address_data)
        conn.commit()
    except IntegrityError as e:
        print("Error", e)
        conn.rollback()  # rollback the transaction if an error occurs 
    finally:
        cursor.close()
else:
    print("No new data to insert into address_location table.")

In [21]:
# hospital table - convert df to list of tuples
hospital_data = [tuple(x) for x in hospital.values]

# SQL query - insert data into hospital table
sql_insert_hospital = """
INSERT INTO hospital (hospital_id, hospital_org_id, name, ein, medicare_provider_number, Address_id, capacity_id, updated_dt)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

# check if insertion is needed
if hospital_data:
    # execute query 
    cursor = conn.cursor()
    try:
        cursor.executemany(sql_insert_hospital, hospital_data)
        conn.commit()
    except IntegrityError as e:
        print("Error", e)
        conn.rollback()  # rollback the transaction if an error 
    finally:
        cursor.close()
else:
    print("No new data to insert into hospital table.")

Error (1062, "Duplicate entry '1' for key 'hospital.PRIMARY'")


In [22]:
# children hospital table - convert df to list of tuples
children_hospital_data = [tuple(x) for x in children_hospital.values]

# SQL query - insert data into children_hospital table
sql_insert_children_hospital = """
INSERT INTO children_hospital(hospital_id, children_hospital_f, updated_dt)
VALUES (%s, %s, %s)
"""

# check if insertion is needed
if children_hospital_data:
    # execute query
    cursor = conn.cursor()
    try:
        cursor.executemany(sql_insert_children_hospital, children_hospital_data)
        conn.commit()
    except IntegrityError as e:
        print("Error", e)
        conn.rollback()  # rollback the transaction if an error occurs 
    finally:
        cursor.close()
else:
    print("No new data to insert into children_hospital table.")

In [23]:
# hospital_capacity table - convert df to list of tuples
hospital_capacity_data = [tuple(x) for x in hospital_capacity.values]

# SQL query - insert data into hospital_capacity table
sql_insert_hospital_capacity = """
INSERT INTO hospital_capacity (capacity_id, hospital_bed_size, hospital_bed_count, updated_dt)
VALUES (%s, %s, %s, %s)
"""

# check if insertion is needed
if hospital_capacity_data:
    # execute query 
    cursor = conn.cursor()
    try:
        cursor.executemany(sql_insert_hospital_capacity, hospital_capacity_data)
        conn.commit()
    except IntegrityError as e:
        print("Error", e)
        conn.rollback()  # rollback the transaction if an error 
    finally:
        cursor.close()
else:
    print("No new data to insert into hospital_capacity table.")

In [24]:
# teaching table - convert df to list of tuples
teaching_data = [tuple(x) for x in teaching.values]

# SQL query - insert data into children_hospital table
sql_insert_teaching = """
INSERT INTO teaching (hospital_id, memb_counc_teach_hosps_f, updated_dt)
VALUES (%s, %s, %s)
"""

# execute query 
if teaching_data:
    # Execute query 
    cursor = conn.cursor()
    try:
        cursor.executemany(sql_insert_teaching, teaching_data)
        conn.commit()
    except IntegrityError as e:
        print("Error", e)
        conn.rollback()  # rollback the transaction if an error occurs 
    finally:
        cursor.close()
else:
    print("No new data to insert into teaching table.")

In [25]:
# church_affiliation table - convert df to list of tuples
church_affiliation_data = [tuple(x) for x in church_affiliation.values]

# SQL query - insert data into church_affiliation table
sql_insert_church_affiliation = """
INSERT INTO church_affiliation (hospital_id, chrch_affl_f, updated_dt)
VALUES (%s, %s, %s)
"""

# execute query 
if church_affiliation_data:
    # Execute query 
    cursor = conn.cursor()
    try:
        cursor.executemany(sql_insert_church_affiliation, church_affiliation_data)
        conn.commit()
    except IntegrityError as e:
        print("Error", e)
        conn.rollback()  # rollback the transaction if an error occurs 
    finally:
        cursor.close()
else:
    print("No new data to insert into church_affiliation table.")

In [26]:
# Urban table - convert df to list of tuples
urban_data = [tuple(x) for x in urban.values]

# SQL query - insert data into address table
sql_insert_urban_data = """
INSERT INTO urban (urban_location_f, hospital_id, Updated_dt)
VALUES (%s, %s, %s)
"""

# execute query 
if urban_data:
    # Execute query 
    cursor = conn.cursor()
    try:
        cursor.executemany(sql_insert_urban_data, urban_data)
        conn.commit()
    except IntegrityError as e:
        print("Error", e)
        conn.rollback()  # rollback the transaction if an error occurs 
    finally:
        cursor.close()
else:
    print("No new data to insert into urban table.")

In [27]:
# Create a cursor
cursor = conn.cursor()

# Execute SQL queries to get table names
cursor.execute("SHOW TABLES")

# Fetch all the tables
tables = cursor.fetchall()

for table in tables:
        table_name = table[0]

        # Execute SQL query to fetch sample data from each table
        cursor.execute(f"SELECT * FROM {table_name} LIMIT 5")

        # Fetch and print sample data
        sample_data = cursor.fetchall()
        print(f"Sample data from table '{table_name}':")
        for row in sample_data:
            print(row)

Sample data from table 'address_location':
(1, '702 Main Street', 'Opp', 'AL', 36462, '01039', 'Covington County', datetime.date(2023, 11, 20))
(2, '50 Medical Park Drive East', 'Birmingham', 'AL', 35235, '01073', 'Jefferson County', datetime.date(2023, 11, 20))
(3, '1000 First Street North', 'Alabaster', 'AL', 35007, '01117', 'Shelby County', datetime.date(2023, 11, 20))
(4, '1720 University Boulevard', 'Birmingham', 'AL', 35233, '01073', 'Jefferson County', datetime.date(2023, 11, 20))
(5, '400 Northwood Drive', 'Centre', 'AL', 35960, '01019', 'Cherokee County', datetime.date(2023, 11, 20))
Sample data from table 'children_hospital':
(46288, 1, b'0', datetime.date(2023, 11, 20))
(46289, 2, b'0', datetime.date(2023, 11, 20))
(46290, 3, b'0', datetime.date(2023, 11, 20))
(46291, 4, b'0', datetime.date(2023, 11, 20))
(46292, 5, b'0', datetime.date(2023, 11, 20))
Sample data from table 'church_affiliation':
(38402, 1, b'0', datetime.date(2023, 11, 20))
(38403, 2, b'0', datetime.date(2023

In [28]:
# bed count view 
create_view_bed_count = """
CREATE OR REPLACE VIEW hospital_bed_count_by_state AS
SELECT al.state, SUM(hc.hospital_bed_count) AS total_bed_count
FROM hospital_capacity hc
JOIN hospital h ON hc.capacity_id = h.capacity_id
JOIN address_location al ON h.address_id = al.address_id
GROUP BY al.state;
"""

try:
    cur = conn.cursor()
    cur.execute(create_view_bed_count)
    conn.commit()
    print("Hospital bed count by state view created successfully.")
except Exception as e:
    print("Error creating hospital bed count by state view:", e)

Hospital bed count by state view created successfully.


In [29]:
# hospital facility view 
create_view_facility_count = """
CREATE OR REPLACE VIEW hospital_facility_count_by_state AS
SELECT al.state, COUNT(*) AS hospital_count
FROM hospital h
JOIN address_location al ON h.address_id = al.address_id
GROUP BY al.state;
"""

try:
    with conn.cursor() as cursor:
        cursor.execute(create_view_facility_count)
        conn.commit()
    print("Hospital facility count by state view created successfully.")
except Exception as e:
    print("Error creating hospital facility count by state view:", e)

Hospital facility count by state view created successfully.


In [30]:
# define function to execute sql query
def execute_query(sql_query):
    try:
        with conn.cursor() as cursor:
            cursor.execute(sql_query)
            rows = cursor.fetchmany(10)
            for row in rows:
                print(row)
    except Exception as e:
        print("Error executing query:", e)

# print queries
print("Viewing hospital_bed_count_by_state:")
execute_query("SELECT * FROM hospital_bed_count_by_state;")

print("\nViewing hospital_facility_count_by_state:")
execute_query("SELECT * FROM hospital_facility_count_by_state;")

Viewing hospital_bed_count_by_state:
('AL', Decimal('12678'))
('AK', Decimal('1616'))
('AZ', Decimal('22406'))
('CA', Decimal('45860'))
('CO', Decimal('10092'))
('CT', Decimal('8595'))
('DE', Decimal('2597'))
('DC', Decimal('2925'))
('FL', Decimal('33736'))
('GA', Decimal('19342'))

Viewing hospital_facility_count_by_state:
('AL', 74)
('AK', 22)
('AZ', 104)
('CA', 208)
('CO', 53)
('CT', 33)
('DE', 7)
('DC', 6)
('FL', 93)
('GA', 98)


## Power Bi Visualizations 

In [31]:
# Power BI Client Integrations Authentication 
device_auth = DeviceCodeLoginAuthentication() # open url and enter code 

Performing device flow authentication. Please follow the instructions below.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code BYYZQ6TVS to authenticate.

Device flow authentication successfully completed.
You are now logged in .

The result should be passed only to trusted code in your notebook.


In [32]:
# SQL query to fetch data for each visualization
hospital_query = "SELECT * FROM hospital"

# load data into df
hospital_df = pd.read_sql(hospital_query, conn)

# create power bi visualization
PBI_visualize_hospital = QuickVisualize(get_dataset_config(hospital_df), auth=device_auth)
PBI_visualize_hospital

QuickVisualize()

In [42]:
# SQL query to fetch data for each visualization
capacity_query = "SELECT * FROM hospital_capacity"

# load data into df
capacity_df = pd.read_sql(capacity_query, conn)

# create power bi visualization
PBI_visualize_capacity = QuickVisualize(get_dataset_config(capacity_df), auth=device_auth)
PBI_visualize_capacity

QuickVisualize()

In [33]:
# SQL query to fetch data for each visualization
bed_count_query = "SELECT * FROM hospital_bed_count_by_state"

# load data into df
bed_count_df = pd.read_sql(bed_count_query, conn)

# create power bi visualization
PBI_visualize_bed_count = QuickVisualize(get_dataset_config(bed_count_df), auth=device_auth)
PBI_visualize_bed_count

QuickVisualize()

In [43]:
# SQL query to fetch data for each visualization
location_query = "SELECT * FROM address_location"

# load data into df
location_df = pd.read_sql(location_query, conn)

# create power bi visualization
PBI_visualize_location = QuickVisualize(get_dataset_config(location_df), auth=device_auth)
PBI_visualize_location

QuickVisualize()

In [38]:
# SQL query to fetch data for each visualization
facility_count_query = "SELECT * FROM hospital_facility_count_by_state"

# load data into df
facility_count_df = pd.read_sql(facility_count_query, conn)

# create power bi visualization
PBI_visualize_facility_count = QuickVisualize(get_dataset_config(facility_count_df), auth=device_auth)
PBI_visualize_facility_count

QuickVisualize()

In [35]:
# initialize the Bing geocoder with your API key
bing_maps_api_key = "AmTKzYBDON0PyUs-5q2tlQytMFCwx8TipIoa72_dt8lMM8K61_C3SlWI6GTmI2JL"
geolocator = Bing(api_key=bing_maps_api_key)

# SQL query to fetch hospital addresses
hospital_query = """
SELECT h.name, al.street_address, al.city, al.state, al.zip_code 
FROM hospital h
JOIN address_location al ON h.address_id = al.address_id;
"""

# fetch hospital data from the db
hospital_data = pd.read_sql(hospital_query, conn)

# perform geocoding for each hospital address to obtain latitude and longitude
for index, row in hospital_data.iterrows():
    address = f"{row['street_address']}, {row['city']}, {row['state']} {row['zip_code']}"
    location = geolocator.geocode(address)
    if location:
        hospital_data.loc[index, 'latitude'] = location.latitude
        hospital_data.loc[index, 'longitude'] = location.longitude

# drop rows with missing coordinates
hospital_data = hospital_data.dropna(subset=['latitude', 'longitude'])

In [36]:
# create map center
hospital_map = folium.Map(location=[hospital_data['latitude'].mean(), hospital_data['longitude'].mean()], zoom_start=3.2)

# add marker for each hospital 
for index, row in hospital_data.iterrows():
    folium.Marker([row['latitude'], row['longitude']], popup=row['name']).add_to(hospital_map)

# Display the map
hospital_map